<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル2

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [ ]:
!pip3 install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00


# 多目的最適化
多目的最適化は最適化する評価値が2つ以上ある最適化です。  
例として以下の数式$f_1, f_2$の最適化をしましょう。
<br />
<br />
$f_1(x,y) = 4x^2+4y~2$

$f_2(x,y)=(x-5)^2 + (y -5)^2$

$(0 \le x \le 5, 0 \le y \le 3)$
<br />
<br />
$f_1$を最小化、$f_2$を最大化する$(x,y)$の探索


In [ ]:
# Optunaをインポート
import optuna


# 目的関数を定義
def f1(x, y):
    return 4 * x**2 + 4 * y**2

def f2(x, y):
    return (x - 5)**2 + (y - 5)**2

def objective(trial): # 引数に　trial を指定
    # サジェストAPIで目的関数への入力を定案
    x = trial.suggest_float("x", 0, 5)
    y = trial.suggest_float("y", 0, 3)

    v1 = f1(x, y)
    v2 = f2(x, y)

    # 変更点1: 目的関数が複数の値を返す
    return v1, v2

# 最適化を実行
study = optuna.create_study(
    # 変更点2: 目的ごとに最適化の方向を指定
    directions=["minimize", "minimize"]
)
study.optimize(objective, n_trials=100)

# 最適化の結果を確認
print("[Best Trials]")
# 変更点3: Study.best_trialのかわりにStudy.best_trialsを使用
for trial in study.best_trials:
    # 変更点4: FrozenTrial.valueのかわりにFrozenTrial.valuesを使用
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")

[I 2023-07-01 09:25:42,165] A new study created in memory with name: no-name-9353233f-73f0-414f-ba46-1be5fd32d913
[I 2023-07-01 09:25:42,172] Trial 0 finished with values: [36.60391845743548, 19.636552119108025] and parameters: {'x': 1.155956252007344, 'y': 2.7954864975177407}. 
[I 2023-07-01 09:25:42,177] Trial 1 finished with values: [38.15248079430721, 18.297138148693453] and parameters: {'x': 1.3430125261008663, 'y': 2.7810856788874685}. 
[I 2023-07-01 09:25:42,180] Trial 2 finished with values: [81.50848273093757, 7.154938723055755] and parameters: {'x': 3.6037710593160623, 'y': 2.7184471366518013}. 
[I 2023-07-01 09:25:42,186] Trial 3 finished with values: [33.0556626581202, 20.467896541264956] and parameters: {'x': 1.1410618972452142, 'y': 2.638540015081295}. 
[I 2023-07-01 09:25:42,191] Trial 4 finished with values: [47.27451923709951, 22.40663843444525] and parameters: {'x': 3.393994188765502, 'y': 0.5472049487174608}. 
[I 2023-07-01 09:25:42,194] Trial 5 finished with values:

[Best Trials]
- [2] params={'x': 3.6037710593160623, 'y': 2.7184471366518013}, values=[81.50848273093757, 7.154938723055755]
- [7] params={'x': 1.663903401055626, 'y': 0.8050948999887756}, values=[13.667009304129664, 28.726769315588403]
- [11] params={'x': 1.415353059827209, 'y': 0.9925153548661548}, values=[11.95324405442933, 28.90962686667369]
- [15] params={'x': 0.9642605238845281, 'y': 1.0197287872353804}, values=[7.878580629754417, 32.12975204623952]
- [16] params={'x': 1.615968127241596, 'y': 1.1439529905361476}, values=[15.679925731269226, 26.320770255039868]
- [20] params={'x': 1.683976309363779, 'y': 1.4510446264909587}, values=[19.765226874266958, 23.59109736001936]
- [21] params={'x': 0.8500627660553739, 'y': 1.9851415035498634}, values=[18.653573981399703, 26.31135079929755]
- [25] params={'x': 2.078206263602265, 'y': 1.5689494969583757}, values=[27.12217519232651, 20.308986192475217]
- [26] params={'x': 1.7558009535329027, 'y': 2.6084157434331723}, values=[39.5466787160683

# 探索の効率化

## 1: 最適化アルゴリズムの選択
### Samperクラス
次に評価すべきパラメータを選択するには、  
サンプラーがパラメータの探索空間から探索点選択することで行われる。

Optunaでは探索点選択を行うアルゴリズムはSamperクラスで実装されている。

<br />

### 探索点選択アルゴリズムの使い分け
以下の公式ドキュメントの記載表より最適化対象で対応しているものを選択する。

最もわかりやすい使い分け基準は、自分の最適化したい目的関数にかけられるトライアル数です。  
例えば、数百トライアル程度ならばデフォルトのTPESamplerを使いますが、  
数千トライアルを実行するときはCmaEsSanplerを利用すべきです。

optuna.samplers — Optuna 3.2.0 documentation:  
https://optuna.readthedocs.io/en/stable/reference/samplers/index.html


|  サンプラー  |  アルゴリズム | 動作速度 | 推奨されるトライアル数 |
| ---- | ---- | ---- | ---- |
|  TPESampler  | ベイズ最適化 | 速い | 最大1000程度 |
|  NSGAIISampler  | 進化計算 | 速い | 最大10000程度 |
| BoTorchSampler | ベイズ最適化 | 遅い | 最大100程度 |
| QMCSampler | 準モンテカルロ法 | 速い | いくらでも |
| cmaEsSampler | 進化計算 | 速い | 最大10000程度 |
| RandomSampler | ランダムサーチ | 速い | いくらでも |
| GridSampler | グリッドサーチ | 速い | 組み合わせの数だけ可能 |

<br />

create_study()の引数で指定  
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html  
単目的最適化のデフォルトは***TPESampler***
多目的最適化のデフォルトは***NGSAIISampler***  

## 2: 枝刈りによるトライアルの早期終了
機械学習では学習時間が長く、1トライアルは設定したエポック分の総学習時間になる。  
そのため枝切りによってハイパーパラメータ調整にかかる実行時間の短縮を行います。  
学習途中の評価位置の変化をみて、望ましい数近いが得られる見込みがない場合は
途中で学習を打ち切ります。  
Optunaでは枝切りを自動的に行うことができます。

<br />

### 枝切りの手順
目的関数の最後に次の手順を追加します。
1.  traial.reportで中間評価値を報告
2.  trial.should_runeで枝刈りを行うべきかの判定

```python
# 1. 中間評価値を報告
trial.report(accuracy, epoch)
# 2. 枝刈りを売るべきか判定する
if trial.should_prune():
  raise optuna.exception.TrialPruned()
```

### prunerコンポーネント
optuna.pruners — Optuna 3.2.0 documentation  
https://optuna.readthedocs.io/en/stable/reference/pruners.html  

枝刈りの機能はprunerというコンポーネントで行われる。  
Optunaではいくつかのprunerが提供されており、
個々のprunerで枝切りの基準が異なる。

create_study()の引数で指定  
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html  

デフォルトは***BasePruner***

### 機械学習における枝切り
枝切りを適応した学習率の最適化では学習率が小さいトライアルが枝切りされやすい傾向になる。  
そのため、枝切りを行わない方設定でprunerで***NopPruner***を用います。

一方で、枝切りを多く行い、トライアル数を増やすことが性能改善につながることもよくあります。  
結論、問題の性質に応じて適したアルゴリズムが変化するので、事前知識が必要です。

## 3: 事前知識の適応
実際のユースケースでは事前に良好な探索点が判明している場合があります。    
全てをOptunaに任せるのではなく、ユーザーの事前知識を活用します。

### ユーザーの事前知識
1.  過去の論文や実験で発見され、良好だと反面しているパラメータ
2.  最適化結果の比較対象（ライブラリのデフォルト値、既存研究や運用中のシステムにおける採用値）が使用しているパラメータ
3. 探索空間中で特に重点的に探索したい領域

### study.enqueue_trialメソッド
enqueueメソッドを使うことによって、サンプラーに探索すべき領域のヒントを与えることができる。





In [ ]:
def objective(trial):
    x = trial.suggest_float("x", -1, 1)
    y = trial.suggest_float("y", -1, 1)
    return x * y

study = optuna.create_study()

study.enqueue_trial({"x": 0.5, "y": -0.3})
study.enqueue_trial({"x": 0.9})

study.optimize(objective, n_trials=3)
for trial in study.trials:
    print(f"[{trial.number}] params={trial.params}, value={trial.value}")